# Import software libraries

In [115]:
# Import required libraries.
import sys           # Read system parameters.
import pandas as pd  # Manipulate and analyze data.
import sqlite3       # Manage SQL databases.

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- pandas {}'.format(pd.__version__))
print('- sqlite3 {}'.format(sqlite3.sqlite_version))

Libraries used in this project:
- Python 3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 23:03:20) 
[GCC 7.3.0]
- pandas 1.1.3
- sqlite3 3.30.1


# Examine the database

In [116]:
# Connect to SQLite database.
conn = sqlite3.connect('/home/jovyan/work/Project/data/prod_sample.db')
conn


In [117]:
# List all the tables in the database.
cursor = conn.cursor()
query = "SELECT name FROM sqlite_master WHERE type='table';"
cursor.execute(query)

# Fetch all the table names
tables = cursor.fetchall()

# Print the list of table names
for table in tables:
    print(table[0])

# Close the cursor and the connection
cursor.close()

stock_description
online_retail_history


# Read data from the `online_retail_history` table

In [118]:
# Write the query to be executed that selects everything from the online_retail_history table.
query = 'SELECT * FROM online_retail_history'



# Use the read_sql function in pandas to read a query into a DataFrame.
DataFrame = pd.read_sql(query, conn)


# Preview the first five rows of the data.
DataFrame[:5]


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50


In [119]:
# Get the shape of the data.
DataFrame.shape


(15321, 8)

# Read data from the `stock_description` table

In [120]:
# Write the query to be executed that selects everything from the online_retail_history table.
query = 'SELECT * FROM stock_description'



# Use the read_sql function in pandas to read a query into a DataFrame.
DataFrame = pd.read_sql(query, conn)


# Preview the first five rows of the data.
DataFrame[:5]

,StockCode,Description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE


In [121]:
# Get the shape of the data.
DataFrame.shape


(3952, 2)

# Aggregate the `online_retail_history` and `stock_description` datasets

In [122]:
# Write a query to aggregate the two datasets so that you have the stock descriptions as well as the stock code.
merge = 'SELECT * FROM stock_description JOIN online_retail_history ON stock_description.StockCode = online_retail_history.StockCode'






# Use the read_sql function in pandas to read a query into a DataFrame.
DataFrame = pd.read_sql(merge, conn)


# Preview the first five rows of the data.
DataFrame[:5]


,StockCode,Description,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,20725,LUNCH BAG RED RETROSPOT,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50
1,20725,LUNCH BAG RED RETROSPOT,536401,20725,1,2010-12-01 11:21:00,1.65,u15862,United Kingdom,1.65
2,20725,LUNCH BAG RED RETROSPOT,536412,20725,1,2010-12-01 11:49:00,1.65,u1792,United Kingdom,1.65
3,20725,LUNCH BAG RED RETROSPOT,536544,20725,4,2010-12-01 14:32:00,4.21,None,United Kingdom,16.84
4,20725,LUNCH BAG RED RETROSPOT,536577,20725,60,2010-12-01 16:13:00,1.45,u13777,United Kingdom,87.00


In [123]:
# Get the shape of the data.
DataFrame.shape


(17032, 10)

# Identify and fix corrupt or unusable data

In [124]:
# Check the value counts of the "Description" field.
DataFrame['Description'].size


17032

In [125]:
# Remove rows where "Description" is just a question mark (?).
DataFrame_cleaned = DataFrame[DataFrame['Description']!= '?']

# Preview the first five rows of the data.
DataFrame_cleaned[:5]


,StockCode,Description,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,20725,LUNCH BAG RED RETROSPOT,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50
1,20725,LUNCH BAG RED RETROSPOT,536401,20725,1,2010-12-01 11:21:00,1.65,u15862,United Kingdom,1.65
2,20725,LUNCH BAG RED RETROSPOT,536412,20725,1,2010-12-01 11:49:00,1.65,u1792,United Kingdom,1.65
3,20725,LUNCH BAG RED RETROSPOT,536544,20725,4,2010-12-01 14:32:00,4.21,None,United Kingdom,16.84
4,20725,LUNCH BAG RED RETROSPOT,536577,20725,60,2010-12-01 16:13:00,1.45,u13777,United Kingdom,87.00


# Identify and remove duplicates

In [126]:
# Identify all duplicated data.
Duplicated = DataFrame_cleaned[DataFrame_cleaned.duplicated(keep=False)]

print('Number of duplicated rows:', Duplicated.shape[0])


Number of duplicated rows: 223


In [127]:
# Print the duplicated data.
print(Duplicated)


      StockCode                         Description Invoice StockCode  \
40        20725             LUNCH BAG RED RETROSPOT  537955     20725   
41        20725             LUNCH BAG RED RETROSPOT  537955     20725   
281       20725             LUNCH BAG RED RETROSPOT  546635     20725   
282       20725             LUNCH BAG RED RETROSPOT  546635     20725   
605       20725             LUNCH BAG RED RETROSPOT  555560     20725   
...         ...                                 ...     ...       ...   
16721    85123A  CREAM HANGING HEART T-LIGHT HOLDER  573543    85123A   
16907    85123A  CREAM HANGING HEART T-LIGHT HOLDER  578255    85123A   
16908    85123A  CREAM HANGING HEART T-LIGHT HOLDER  578255    85123A   
16952    85123A  CREAM HANGING HEART T-LIGHT HOLDER  579458    85123A   
16953    85123A  CREAM HANGING HEART T-LIGHT HOLDER  579458    85123A   

       Quantity          InvoiceDate  Price CustomerID         Country  \
40            1  2010-12-09 11:28:00   1.65     u

In [128]:
# Remove the duplicated data.
DataFrame_Cleaned_nodups = DataFrame_cleaned.drop_duplicates()


# Preview the first five rows of the data.
DataFrame_Cleaned_nodups[:5]


,StockCode,Description,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,20725,LUNCH BAG RED RETROSPOT,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50
1,20725,LUNCH BAG RED RETROSPOT,536401,20725,1,2010-12-01 11:21:00,1.65,u15862,United Kingdom,1.65
2,20725,LUNCH BAG RED RETROSPOT,536412,20725,1,2010-12-01 11:49:00,1.65,u1792,United Kingdom,1.65
3,20725,LUNCH BAG RED RETROSPOT,536544,20725,4,2010-12-01 14:32:00,4.21,None,United Kingdom,16.84
4,20725,LUNCH BAG RED RETROSPOT,536577,20725,60,2010-12-01 16:13:00,1.45,u13777,United Kingdom,87.00


# Correct date formats

In [129]:
# Get the data types for every column in the DataFrame.
print(DataFrame_Cleaned_nodups.dtypes)


StockCode       object
Description     object
Invoice         object
StockCode       object
Quantity         int64
InvoiceDate     object
Price          float64
CustomerID      object
Country         object
TotalAmount    float64
dtype: object


In [130]:
# Convert "InvoiceDate" to a "%Y-%m-%d" datetime format.
DataFrame_Cleaned_nodups['InvoiceDate'] = pd.to_datetime(DataFrame_Cleaned_nodups['InvoiceDate'], format="%Y-%m-%d")



In [131]:
# Get the data types for every column in the converted DataFrame.
print(DataFrame_Cleaned_nodups.dtypes)



StockCode              object
Description            object
Invoice                object
StockCode              object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
CustomerID             object
Country                object
TotalAmount           float64
dtype: object


# Examine the table before finishing

In [132]:
# Preview the first five rows of the data.
DataFrame_Final = DataFrame_Cleaned_nodups
DataFrame_Final[:5]

,StockCode,Description,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,20725,LUNCH BAG RED RETROSPOT,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50
1,20725,LUNCH BAG RED RETROSPOT,536401,20725,1,2010-12-01 11:21:00,1.65,u15862,United Kingdom,1.65
2,20725,LUNCH BAG RED RETROSPOT,536412,20725,1,2010-12-01 11:49:00,1.65,u1792,United Kingdom,1.65
3,20725,LUNCH BAG RED RETROSPOT,536544,20725,4,2010-12-01 14:32:00,4.21,None,United Kingdom,16.84
4,20725,LUNCH BAG RED RETROSPOT,536577,20725,60,2010-12-01 16:13:00,1.45,u13777,United Kingdom,87.00


# Load the dataset into a pickle file

In [133]:
# Save the dataset as a pickle file named online_history_cleaned.pickle.
DataFrame_Final.to_pickle('online_history_cleaned.pickle')


In [134]:
# Close any connections to the database.
cursor.close()
